Bienvenido a la nueva version de 

# Chasing Squares-MoeViews 3D
                                                
Aqui se pueden visualizar las imagenes de la 'funcion de Vaughan' en la banda de Moebius tridimensional.

In [ ]:
'PACKAGES'
import numpy as np
import cmath
from shapely.geometry import Polygon, LineString
from numpy import asarray
from numpy import savetxt
import ipyvolume as ipv
import pickle
import time
import os
import matplotlib
matplotlib.use('qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import cm
plt.ion()


In [ ]:
#lOAD PICKLED FILES
        
cho = int(input('In which computer are we working right now?:\n For Ulises\' PC type 0 \n For Cristina\'s PC type 1\n '))
CristinaPath = os.path.join('C:\\','Users', 'crivi', 'OneDrive', 'Desktop', 'CurvasIpy', 'PickledCurves')
UlisesPath = os.path.join('C:\\','Users', 'ulise', 'Documents', 'ChasingSquares', 'PickledCurves')
if cho==0: 
    ChosenPath = UlisesPath
if cho==1:
    ChosenPath = CristinaPath
else:
    print("Run the cell again and type 0 or 1")
    
NewCurve=str(input("OK!\n And what\'s the name of the directory you want to load the files from? : "))

#direcciones de archivos en directorio
dirRojaM3D = os.path.join(ChosenPath, NewCurve,'roja90M3DPickle' )
dirAzulM3D = os.path.join(ChosenPath, NewCurve,'azul180M3DPickle')
dirCurvar = os.path.join(ChosenPath, NewCurve,'curvarPickle')   
dirAngIn = os.path.join(ChosenPath, NewCurve,'angin.txt') 
dirAngFi = os.path.join(ChosenPath, NewCurve,'angfi.txt') 
dirPasito = os.path.join(ChosenPath, NewCurve,'pasito.txt') 
  
#Revisar si los directorios existen  
AllFilesExist=np.zeros(6)
if os.path.exists(dirRojaM3D):
    print(dirRojaM3D + ' : exists')
    AllFilesExist[0]=1
if os.path.exists(dirAzulM3D):
    print(dirAzulM3D + ' : exists')
    AllFilesExist[1]=1
if os.path.exists(dirCurvar):
    print(dirCurvar + ' : exists')
    AllFilesExist[2]=1
if os.path.exists(dirAngIn):
    print(dirAngIn + ' : exists')
    AllFilesExist[3]=1
if os.path.exists(dirAngFi):
    print(dirAngFi + ' : exists')
    AllFilesExist[4]=1
if os.path.exists(dirPasito):
    print(dirPasito + ' : exists')
    AllFilesExist[5]=1
    
if np.sum(AllFilesExist)==6:
    readroja90M3D = open(dirRojaM3D, "rb")
    roja90 = pickle.load(readroja90M3D)
    readroja90M3D.close()  
    
    readazul180M3D = open(dirAzulM3D, "rb")
    azul180 = pickle.load(readazul180M3D)
    readazul180M3D.close()
                   
    readcurvar = open(dirCurvar, "rb")
    curvar = pickle.load(readcurvar)
    readcurvar.close()
    
    readangin = open(dirAngIn, "rb")
    deg = pickle.load(readangin)
    readangin.close()
    
    readangfi = open(dirAngFi, "rb")
    noa = pickle.load(readangfi)
    readangfi.close()
    
    readpasito = open(dirPasito, "rb")
    pasito = pickle.load(readpasito)
    readpasito.close()
    
    print("All done! your variables are loaded")

if np.array_equal(AllFilesExist,[1,1,1,0,0,0]) == True:
    readroja90M3D = open(dirRojaM3D, "rb")
    roja90 = pickle.load(readroja90M3D)
    readroja90M3D.close()  
    
    readazul180M3D = open(dirAzulM3D, "rb")
    azul180 = pickle.load(readazul180M3D)
    readazul180M3D.close()
                   
    readcurvar = open(dirCurvar, "rb")
    curvar = pickle.load(readcurvar)
    readcurvar.close()
    
    deg=0
    noa=90
    pasito=1
    
    print("All done! your variables are loaded")

else:
    print("THERE IS A PROBLEM: Can\'t find the files!")


In [ ]:
#CALCULO COORDENADAS SUPERFICIE BANDA MOEBIUS 3D  

bandam3d=[]
banda90m3d=[]
densbanda=300
for rad in np.linspace(1,3,densbanda,endpoint=True):
    bandam3d=[]
    for alpha in np.linspace(0,2*np.pi,densbanda,endpoint=True):
        z1=cmath.rect(rad,alpha)
        if cmath.phase(z1) >= 0:
            z=z1
            z = (z**2)/(abs(z))
            if np.angle(z)>=0:
                teta = np.angle(z)
            else:
                teta= 2*np.pi+cmath.phase(z)
            v = cmath.rect(1, -teta/2 + np.pi/2)
            w = (3-abs(z))*v
            z = cmath.rect(3,teta)+cmath.rect(w.real,teta)
            zh = complex(w.imag,0)
            bandam3d = np.append(bandam3d, [z,zh])
        if cmath.phase(z1) < 0:
            z=z1
            z = z*cmath.rect(1,np.pi)
            z = (z**2)/(abs(z))
            if np.angle(z)>=0:
                teta = np.angle(z)
            else:
                teta= 2*np.pi+cmath.phase(z)
            v = cmath.rect(1, -teta/2 + 3*np.pi/2)
            w = (3-abs(z))*v            
            z = cmath.rect(3,teta)+cmath.rect(w.real,teta)
            zh = complex(w.imag,0)
            bandam3d = np.append(bandam3d, [z, zh])
            
    banda90m3d = banda90m3d+[bandam3d]


In [ ]:
#DEFINICION DEL GRID COORDENADAS POLARES

# Grid circulos cada 0.5
circulog=np.array([],dtype=complex)
rads=np.arange(1, 4, 2)
for rad in rads:
    for i in range (0,181):
        cig=cmath.rect(rad,(2*(np.pi)*i)/180)
        circulog=np.append(circulog, cig) 
        
# Grid angulos cada 10 grados
griddeg=np.array([],dtype=complex)
angulog= np.arange(0, 180,10)
for gdeg in angulog:
    gangulo = (gdeg)*(np.pi)/180
    ga1=cmath.rect(3,gangulo)
    ga2=cmath.rect(3,gangulo+(np.pi))
    griddeg=np.append(griddeg,[ga1,ga2,ga1])

In [ ]:
# COORDENADAS EJES 
eje1=[]
eje2=[]
beje = np.arange(-5, 5, 0.25)
grideje=np.array(beje,dtype=complex)

for degr in np.arange(0,90):
    anguli = (degr)*(np.pi)/180
    rota1=cmath.rect(1,anguli)
    rota2=cmath.rect(1,anguli+np.pi/2)
    rotaeje1=np.dot(grideje,rota1)
    rotaeje2=np.dot(grideje,rota2)
    eje1=eje1+[rotaeje1]
    eje2=eje2+[rotaeje2]

In [ ]:
#DEFINICION DE COORDENADAS PARA ANIMACION 

lenangulitos = len(np.arange(deg,deg+noa,pasito))
angulog = np.arange(0,lenangulitos)
rban = np.arange(0,len(banda90m3d))

#Banda 3D Surface:
xb = np.array([(banda90m3d[s][::2].real) for s in rban])
yb = np.array([(banda90m3d[s][::2].imag) for s in rban])
zb = np.array([(banda90m3d[s][1::2].real) for s in rban])
#Roja(deg, deg+noa):
xr = np.array([(roja90[t][::2].real) for t in angulog])
yr = np.array([(roja90[t][::2].imag) for t in angulog])
zr = np.array([(roja90[t][1::2].real) for t in angulog])

#Azul(deg, deg+noa):
xa = np.array([(azul180[t][::2].real) for t in angulog])
ya = np.array([(azul180[t][::2].imag) for t in angulog])
za = np.array([(azul180[t][1::2].real) for t in angulog])

#Eje 1 (deg, deg+noa):
xe1 = np.array([(eje1[t+deg].real) for t in angulog])
ye1 = np.array([(eje1[t+deg].imag) for t in angulog])
sha1=xe1.shape
ze1 = np.zeros(sha1)

#Eje 2 (deg, deg+noa):
xe2 = np.array([(eje2[t+deg].real) for t in angulog])
ye2 = np.array([(eje2[t+deg].imag) for t in angulog])
sha2=xe2.shape
ze2 = np.zeros(sha2)

#ANIMACION 360 SOLO EN CASO DE QUE SE CUBRAN TODOS LOS ANGULOS
if noa-deg == 90:
    #Roja(0-360):
    xr = np.append(xr,xa, axis=0)
    yr = np.append(yr,ya, axis=0)
    zr = np.append(zr,za, axis=0)

    #Azul(180-180+360)
    xa = np.append(xa,xr, axis=0)
    ya = np.append(ya,yr, axis=0)
    za = np.append(za,zr, axis=0)

    #Eje 1 (0-360)
    xe1 = np.append(xe1,xe2, axis=0)
    ye1 = np.append(ye1,ye2, axis=0)
    sha1 = xe1.shape
    ze1 = np.zeros(sha1) 

    #Eje 2 (180-180+360)
    xe2 = np.append(xe2,xe1,axis=0)
    ye2 = np.append(ye2,ye1,axis=0)
    sha2 = xe1.shape
    ze2 = np.zeros(sha2) 

In [ ]:
#ANIMACION 
nov=len(curvar)
zzz=np.zeros(nov)
zz1=np.zeros(len(circulog))
zz0=np.zeros(len(griddeg))
angucolor=np.arange(0,540,10)
colormap=cm.twilight
colora=colormap(angucolor)

anim360=ipv.figure()
#ipv.pylab.plot(griddeg.real, griddeg.imag, zz0, color=colora)
ipv.plot_surface(xb, yb, zb, color="gray")
ipv.pylab.plot(circulog.real, circulog.imag, zz1, color='gray')
ipv.pylab.plot(curvar.real, curvar.imag, zzz, color='green', linewidth=19, markersize=12)
eje1R = ipv.scatter(xe1, ye1, ze1, color='orange', marker='arrow')
eje2A =ipv.scatter(xe2, ye2, ze2, color='cyan', marker='arrow')
ro1 = ipv.scatter(xr, yr, zr, size=0.5, marker="sphere", grow_limits=False)
az1 = ipv.scatter(xa, ya, za, size=0.5, color="blue", marker="sphere", grow_limits=False) 
ipv.animation_control([ro1,az1,eje1R,eje2A], interval=45)
ipv.style.use("dark")
ipv.xlim(-5,5)
ipv.ylim(-5,5)
ipv.zlim(-5,5)
ipv.show()
anim360.animation = 0